In [1]:
import pandas as pd
import sqlite3

In [2]:
pd.options.display.max_columns = None

In [3]:
conn = sqlite3.connect('sakila.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
SELECT * FROM sqlite_master WHERE type = 'table'
''')
data = cursor.fetchall()
columns = [col[0] for col in cursor.description]
cursor.close()

In [5]:
data_df = pd.DataFrame(data, columns=columns)
data_df

,type,name,tbl_name,rootpage,sql
0,table,actor,actor,2,"CREATE TABLE ""actor"" (\n""index"" INTEGER,\n ""a..."
1,table,store,store,7,"CREATE TABLE ""store"" (\n""index"" INTEGER,\n ""s..."
2,table,address,address,9,"CREATE TABLE ""address"" (\n""index"" INTEGER,\n ..."
3,table,category,category,26,"CREATE TABLE ""category"" (\n""index"" INTEGER,\n ..."
4,table,city,city,28,"CREATE TABLE ""city"" (\n""index"" INTEGER,\n ""ci..."
5,table,country,country,39,"CREATE TABLE ""country"" (\n""index"" INTEGER,\n ..."
6,table,customer,customer,43,"CREATE TABLE ""customer"" (\n""index"" INTEGER,\n ..."
7,table,film_actor,film_actor,63,"CREATE TABLE ""film_actor"" (\n""index"" INTEGER,\..."
8,table,film_category,film_category,126,"CREATE TABLE ""film_category"" (\n""index"" INTEGE..."
9,table,inventory,inventory,140,"CREATE TABLE ""inventory"" (\n""index"" INTEGER,\n..."


Task 1:

Let's start with creating a table that provides the following details: actor's first and last name combined as full_name, film title, film description and length of the movie.

How many rows are there in the table?

In [20]:
pd.read_sql_query(sql = '''
SELECT first_name || ' ' || last_name AS full_name, title, description, length
FROM actor a
JOIN film_actor fa
ON a.actor_id = fa.actor_id
JOIN film f
ON fa.film_id = f.film_id;
''', con = conn)

,full_name,title,description,length
0,Penelope Guiness,Academy Dinosaur,A Epic Drama of a Feminist And a Mad Scientist...,86
1,Penelope Guiness,Anaconda Confessions,A Lacklusture Display of a Dentist And a Denti...,92
2,Penelope Guiness,Angels Life,A Thoughtful Display of a Woman And a Astronau...,74
3,Penelope Guiness,Bulworth Commandments,A Amazing Display of a Mad Cow And a Pioneer w...,61
4,Penelope Guiness,Cheaper Clyde,A Emotional Character Study of a Pioneer And a...,87
...,...,...,...,...
5457,Thora Temple,Telegraph Voyage,A Fateful Yarn of a Husband And a Dog who must...,148
5458,Thora Temple,Trojan Tomorrow,A Astounding Panorama of a Husband And a Sumo ...,52
5459,Thora Temple,Virginian Pluto,A Emotional Panorama of a Dentist And a Crocod...,164
5460,Thora Temple,Wardrobe Phantom,A Action-Packed Display of a Mad Cow And a Ast...,178


Solution:

SELECT a.first_name, 
       a.last_name,
       a.first_name || ' ' || a.last_name AS full_name,
       f.title,
       f.length
FROM   film_actor fa
JOIN   actor a
ON     fa.actor_id = a.actor_id
JOIN   film f
ON     f.film_id = fa.film_id

Task 2:

Write a query that creates a list of actors and movies where the movie length was more than 60 minutes. How many rows are there in this query result?

In [21]:
pd.read_sql_query(sql = '''
SELECT first_name || ' ' || last_name AS full_name, title, description, length
FROM actor a
JOIN film_actor fa
ON a.actor_id = fa.actor_id
JOIN film f
ON fa.film_id = f.film_id
WHERE length > 60;
''', con = conn)

,full_name,title,description,length
0,Alec Wayne,Chamber Italian,A Fateful Reflection of a Moose And a Husband ...,117
1,Henry Berry,Chamber Italian,A Fateful Reflection of a Moose And a Husband ...,117
2,Rip Winslet,Chamber Italian,A Fateful Reflection of a Moose And a Husband ...,117
3,Gina Degeneres,Chamber Italian,A Fateful Reflection of a Moose And a Husband ...,117
4,Adam Hopper,Chamber Italian,A Fateful Reflection of a Moose And a Husband ...,117
...,...,...,...,...
4895,Carmen Hunt,Zoolander Fiction,A Fateful Reflection of a Waitress And a Boat ...,101
4896,Mary Tandy,Zoolander Fiction,A Fateful Reflection of a Waitress And a Boat ...,101
4897,Penelope Cronyn,Zoolander Fiction,A Fateful Reflection of a Waitress And a Boat ...,101
4898,Whoopi Hurt,Zoolander Fiction,A Fateful Reflection of a Waitress And a Boat ...,101


Solution:

SELECT a.first_name, 
       a.last_name,
       a.first_name || ' ' || a.last_name AS full_name,
       f.title ,
       f.length
FROM   film_actor fa
JOIN   actor a
ON     fa.actor_id = a.actor_id
JOIN   film f
ON     f.film_id = fa.film_id
WHERE  f.length > 60

Task 3:

Write a query that captures the actor id, full name of the actor, and counts the number of movies each actor has made. (HINT: Think about whether you should group by actor id or the full name of the actor.) Identify the actor who has made the maximum number movies.

In [23]:
pd.read_sql_query(sql = '''
SELECT fa.actor_id, first_name || ' ' || last_name AS full_name, COUNT (*) count
FROM actor a
JOIN film_actor fa
ON a.actor_id = fa.actor_id
JOIN film f
ON fa.film_id = f.film_id
GROUP BY fa.actor_id
ORDER BY count DESC;
''', con = conn)

,actor_id,full_name,count
0,107,Gina Degeneres,42
1,102,Walter Torn,41
2,198,Mary Keitel,40
3,181,Matthew Carrey,39
4,23,Sandra Kilmer,37
...,...,...,...
195,31,Sissy Sobieski,18
196,186,Julia Zellweger,16
197,199,Julia Fawcett,15
198,35,Judy Dean,15


Solution:

SELECT actorid, full_name, 
       COUNT(filmtitle) film_count_peractor
FROM
    (SELECT a.actor_id actorid,
            a.first_name, 
            a.last_name,
            a.first_name || ' ' || a.last_name AS full_name,
            f.title filmtitle
    FROM    film_actor fa
    JOIN    actor a
    ON      fa.actor_id = a.actor_id
    JOIN    film f
    ON      f.film_id = fa.film_id) t1
GROUP BY 1, 2
ORDER BY 3 DESC

In [22]:
pd.read_sql_query(sql = '''
SELECT actorid, full_name, 
       COUNT(filmtitle) film_count_peractor
FROM
    (SELECT a.actor_id actorid,
            a.first_name, 
            a.last_name,
            a.first_name || ' ' || a.last_name AS full_name,
            f.title filmtitle
    FROM    film_actor fa
    JOIN    actor a
    ON      fa.actor_id = a.actor_id
    JOIN    film f
    ON      f.film_id = fa.film_id) t1
GROUP BY 1, 2
ORDER BY 3 DESC;
''', con = conn)

,actorid,full_name,film_count_peractor
0,107,Gina Degeneres,42
1,102,Walter Torn,41
2,198,Mary Keitel,40
3,181,Matthew Carrey,39
4,23,Sandra Kilmer,37
...,...,...,...
195,71,Adam Grant,18
196,186,Julia Zellweger,16
197,35,Judy Dean,15
198,199,Julia Fawcett,15
